# Who will exit first?



In [1]:
import pandas as pd

# read data
dataset = pd.read_csv("Churn_Modelling.csv", header=0)
dataset.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [5]:
# Unique geographic values in dataset
if dataset["Gender"][0] in ("Male", "Female"):
    unique_geography = dataset[["Geography"]].groupby("Geography").size()
print(unique_geography)

Geography
France     5014
Germany    2509
Spain      2477
dtype: int64


Transformation of some values in dataset.

In [6]:
# transform gender values geographic values
if dataset["Gender"][0] in ("Male", "Female"):
    for row in [dataset]:
        row["Gender"] = row["Gender"].map( {"Male": 0, "Female": 1} ).astype(int)
        row["Geography"] = row["Geography"].map( {"France": 0, "Germany": 1, "Spain": 2} ).astype(int)
    # end of for loop
# end of if clause

# transform age values
if dataset["Age"][0] > 17:
    dataset.loc[ dataset["Age"] <= 16, "Age"] = 0
    dataset.loc[(dataset["Age"] > 16) & (dataset["Age"] <= 32), "Age"] = 1
    dataset.loc[(dataset["Age"] > 32) & (dataset["Age"] <= 48), "Age"] = 2
    dataset.loc[(dataset["Age"] > 48) & (dataset["Age"] <= 64), "Age"] = 3
    dataset.loc[ dataset["Age"] > 64, "Age"] = 4
# end of if clause

if "RowNumber" in dataset.columns:
    dataset = dataset.drop(["RowNumber"], axis=1)
if "CustomerId" in dataset.columns:
    dataset = dataset.drop(["CustomerId"], axis=1)

dataset.describe()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,0.746300,0.454300,1.903400,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,0.827529,0.497932,0.713105,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,0.000000,0.000000,1.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,0.000000,0.000000,2.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,1.000000,1.000000,2.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,2.000000,1.000000,4.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Extracting train dataset from dataset.

In [7]:
test_indis = 10

train_dataset = dataset[dataset.index % test_indis != 0]
train_dataset.describe()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000
mean,650.150222,0.747222,0.453222,1.906444,5.003333,76374.634737,1.526778,0.705444,0.514778,99980.811722,0.205556
std,96.594345,0.829365,0.497835,0.713655,2.892102,62371.972554,0.578492,0.455868,0.499809,57593.448587,0.404129
min,350.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,583.000000,0.000000,0.000000,1.000000,2.000000,0.000000,1.000000,0.000000,0.000000,50768.320000,0.000000
50%,651.000000,0.000000,0.000000,2.000000,5.000000,96999.225000,1.000000,1.000000,1.000000,100272.165000,0.000000
75%,717.000000,1.000000,1.000000,2.000000,7.000000,127574.692500,2.000000,1.000000,1.000000,149416.395000,0.000000
max,850.000000,2.000000,1.000000,4.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000,1.000000


In [8]:
describe = train_dataset.describe()
train_dataset.mean()

CreditScore          650.150222
Geography              0.747222
Gender                 0.453222
Age                    1.906444
Tenure                 5.003333
Balance            76374.634737
NumOfProducts          1.526778
HasCrCard              0.705444
IsActiveMember         0.514778
EstimatedSalary    99980.811722
Exited                 0.205556
dtype: float64

Extracting test dataset from dataset

In [9]:
test_dataset = dataset[dataset.index % test_indis == 0]
test_dataset.describe()

#total count of sample space
#total = float(len(train_dataset))

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.00000,1000.000000,1000.000000,1000.000000
mean,653.936000,0.738000,0.464000,1.876000,5.098000,77487.180250,1.56100,0.70600,0.518000,101075.093310,0.187000
std,97.164422,0.811181,0.498952,0.707902,2.892882,62648.246306,0.60881,0.45582,0.499926,56777.420486,0.390107
min,382.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.00000,0.00000,0.000000,91.750000,0.000000
25%,588.000000,0.000000,0.000000,1.000000,3.000000,0.000000,1.00000,0.00000,0.000000,53467.967500,0.000000
50%,655.000000,1.000000,0.000000,2.000000,5.000000,98409.815000,2.00000,1.00000,1.000000,99946.705000,0.000000
75%,720.250000,1.000000,1.000000,2.000000,8.000000,128771.470000,2.00000,1.00000,1.000000,148629.425000,0.000000
max,850.000000,2.000000,1.000000,4.000000,10.000000,238387.560000,4.00000,1.00000,1.000000,199970.740000,1.000000
